In [6]:
import pandas_gbq
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.decomposition import PCA
import pickle
import h5py
warnings.filterwarnings("ignore")

In [7]:
with open('../../app/SQL/Query_traer_clientes_simplificado.sql') as file:
    query=file.read()
data=pandas_gbq.read_gbq(query, project_id='customer-experience-384423',progress_bar_type=None)

In [9]:
data.describe()

,id_cliente,promedio_compra,cantidad_compras,dropsize,contactable,compro_en_OUTLET,compro_en_ESTELAR,compro_en_MOVIES,compro_en_LITTLE_MIC,compro_en_MOVIES_W,...,compro_en_ZONA_4,compro_en_ZONA_5,compro_en_ZONA_6,compro_en_ZONA_7,compro_en_ZONA_M,compro_en_ECUADOR,compras_2018,compras_2019,compras_2020,compras_2021
count,250209.0,2.502090e+05,250209.0,2.502090e+05,250209.0,250209.0,250209.0,250209.0,250209.0,250209.0,...,250209.0,250209.0,250209.0,250209.0,250209.0,250209.0,8.375000e+03,1.600100e+04,4.145100e+04,1.229280e+05
mean,125105.999996,7.746694e+16,4.189805,3.043208e+16,0.579476,0.0,0.0,0.0,0.846197,0.0,...,0.181452,0.086716,0.153843,0.103885,0.0,0.0,3.127884e+05,3.400870e+05,2.758118e+18,4.426974e+05
std,72229.261096,5.908390e+17,14.888362,3.181584e+17,0.493644,0.0,0.0,0.0,0.360761,0.0,...,0.385393,0.281418,0.360799,0.305112,0.0,0.0,8.146475e+05,1.768256e+06,9.693510e+18,1.760022e+06
min,1.0,-4.644237e+05,1.0,-3.505040e+05,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-5.312610e+05,-3.240320e+05,-7.352690e+05,-2.136349e+07
25%,62554.0,4.293500e+04,1.0,1.824268e+04,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.193300e+04,4.699200e+04,5.285700e+04,5.453800e+04
50%,125106.0,6.561740e+04,2.0,2.814700e+04,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.021840e+05,1.156310e+05,1.895170e+05,1.523190e+05
75%,187658.0,1.058650e+05,5.0,4.074800e+04,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.765550e+05,3.024700e+05,1.352156e+06,4.033080e+05
max,250210.0,9.998460e+18,6803.0,9.908710e+18,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,0.0,0.0,1.822834e+07,1.905567e+08,4.019241e+20,3.226566e+08


## CLIENTES INSATISFECHOS POR ZONA

Verificar la cantidad de clientes cuyo promedio de compras es negativo en un período es importante para una la industria textil para:

1. Identificar problemas en la satisfacción del cliente 
2. problemas de calidad
3. servicio al cliente deficiente
4. incumplimiento de expectativas

Verificar esta situación ayudaría a identificar los problemas específicos y tomar medidas para mejorar la satisfacción del cliente.

Tambien sirve para evaluar la salud financiera de la empresa y tomar decisiones informadas sobre estrategias de marketing y retención de clientes

Se encontraron 224 personas de este grupo que son contactables y que se les deberia hacer un seguimiento para mejorar la satisfación de los clientes

In [51]:
data_promedio_negativo = data[data['promedio_compra'] < 0]
devoluciones={}
devoluciones['clientes_zona1']=len(data_promedio_negativo[data_promedio_negativo['compro_en_ZONA_1']==1])
devoluciones['clientes_zona2']=len(data_promedio_negativo[data_promedio_negativo['compro_en_ZONA_2']==1])
devoluciones['clientes_zona3']=len(data_promedio_negativo[data_promedio_negativo['compro_en_ZONA_3']==1])
devoluciones['clientes_zona4']=len(data_promedio_negativo[data_promedio_negativo['compro_en_ZONA_4']==1])
devoluciones['clientes_zona5']=len(data_promedio_negativo[data_promedio_negativo['compro_en_ZONA_5']==1])
devoluciones['clientes_zona6']=len(data_promedio_negativo[data_promedio_negativo['compro_en_ZONA_6']==1])
devoluciones['clientes_zona7']=len(data_promedio_negativo[data_promedio_negativo['compro_en_ZONA_7']==1])
pd.DataFrame.from_dict(devoluciones, orient='index',columns=['cantidad_clientes_insatisfechos'])

,cantidad_clientes_insatisfechos
clientes_zona1,19
clientes_zona2,40
clientes_zona3,182
clientes_zona4,104
clientes_zona5,51
clientes_zona6,40
clientes_zona7,34


In [53]:
len(data[(data['promedio_compra'] < 0)& (data['contactable'] > 0)])

224

## CANTIDAD DE CLIENTES POR ZONA QUE HAN AUMENTADO SUS COMPRAS EN 2021 CON RESPECTO A 2020


Verificar la cantidad de clientes que han aumentado sus compras con respecto al período anterior es importante para la industria textil para:

1. Evaluar el crecimiento y éxito del negocio 
2. satisfacción del cliente
3. objetivos de crecimiento
4. nuevas oportunidades de negocio para aplicar nuevas campañas
5. requerir recursos y capacidades adicionales para satisfacer la demanda



In [28]:
data['aumento_compras'] = data['compras_2021'] - data['compras_2020']
data_aumento_positivo = data[data['aumento_compras'] > 0]
aumento_positivo_clientes={}
aumento_positivo_clientes['clientes_zona1']=len(data_aumento_positivo[data_aumento_positivo['compro_en_ZONA_1']==1])
aumento_positivo_clientes['clientes_zona2']=len(data_aumento_positivo[data_aumento_positivo['compro_en_ZONA_2']==1])
aumento_positivo_clientes['clientes_zona3']=len(data_aumento_positivo[data_aumento_positivo['compro_en_ZONA_3']==1])
aumento_positivo_clientes['clientes_zona4']=len(data_aumento_positivo[data_aumento_positivo['compro_en_ZONA_4']==1])
aumento_positivo_clientes['clientes_zona5']=len(data_aumento_positivo[data_aumento_positivo['compro_en_ZONA_5']==1])
aumento_positivo_clientes['clientes_zona6']=len(data_aumento_positivo[data_aumento_positivo['compro_en_ZONA_6']==1])
aumento_positivo_clientes['clientes_zona7']=len(data_aumento_positivo[data_aumento_positivo['compro_en_ZONA_7']==1])
pd.DataFrame.from_dict(aumento_positivo_clientes, orient='index',columns=['cantidad_clientes_positivos'])

,cantidad_clientes_positivos
clientes_zona1,374
clientes_zona2,852
clientes_zona3,1786
clientes_zona4,1321
clientes_zona5,521
clientes_zona6,878
clientes_zona7,297


## cantidad de personas que han aumentado sus ventas  y que son contactables

In [33]:
len(data_aumento_positivo['contactable']==1)

5042

In [36]:
data_aumento_positivo_contactable=data_aumento_positivo[data_aumento_positivo['contactable']==1]

In [47]:
lista_zonas=['compro_en_ZONA_1','compro_en_ZONA_2','compro_en_ZONA_3','compro_en_ZONA_4','compro_en_ZONA_5','compro_en_ZONA_6','compro_en_ZONA_7']
lista_productos=['ELEFANTES','PELUCHES','CALCETINES','PELUCHE','BOLSO','CARTUCHERA','BILLETERA','FUNKOS','SALIDA_DE_BANO','GAFAS','SET_DE_ALIMENTACION','JUGUETERIA','CARTERA','MALETA_VIAJERA','TENIS_CON_RUEDAS','SET_DE_BELLEZA','SUECOS','MORRAL','PANTUFLAS','PARAGUAS','LONCHERA','SANDALIAS','BOTAS','BONO','MALETA_CON_RUEDAS','SET_ESCOLAR','GORRAS','GORRO','COBIJA','BABERO','TENIS','BALETA','ZAPATOS','TENIS_CON_LUCES','CORREAS','AROMAS','POP','BOLSA','RELOJ','MOCHILA','ESTUCHES','PANALERA','DULCES','CAMISETA','SET','STICKERS','PANTALON','ACCESORIOS']

cantidad_compras_zona = pd.DataFrame(columns=lista_zonas, index=lista_productos)

for zona in lista_zonas:
    for producto in lista_productos:
        cantidad = ((data_aumento_positivo_contactable[producto] > 0) & (data_aumento_positivo_contactable[zona] > 0)).sum()
        cantidad_compras_zona[zona][producto] = cantidad
conclusion_final=cantidad_compras_zona.loc[~(cantidad_compras_zona == 0).all(axis=1)]
conclusion_final

,compro_en_ZONA_1,compro_en_ZONA_2,compro_en_ZONA_3,compro_en_ZONA_4,compro_en_ZONA_5,compro_en_ZONA_6,compro_en_ZONA_7
PELUCHES,12,12,30,30,3,23,10
CALCETINES,8,34,47,42,12,11,24
PELUCHE,0,0,0,0,0,1,1
GAFAS,14,15,66,58,12,28,17
SUECOS,8,16,40,36,7,27,12
MORRAL,1,2,3,10,1,2,1
PANTUFLAS,5,11,18,20,4,9,10
LONCHERA,5,5,9,5,1,6,1
SANDALIAS,28,58,100,97,15,67,22
BOTAS,0,1,1,1,0,0,0


In [48]:
conclusion_final.sum(axis=1)

PELUCHES       120
CALCETINES     178
PELUCHE          2
GAFAS          210
SUECOS         146
MORRAL          20
PANTUFLAS       77
LONCHERA        32
SANDALIAS      387
BOTAS            3
GORRAS         180
GORRO           42
COBIJA           2
BABERO          10
TENIS           55
BALETA           4
ZAPATOS          2
POP              6
CAMISETA      3252
PANTALON        83
dtype: object

In [49]:
conclusion_final.sum(axis=0)

compro_en_ZONA_1     292
compro_en_ZONA_2     612
compro_en_ZONA_3    1512
compro_en_ZONA_4    1048
compro_en_ZONA_5     348
compro_en_ZONA_6     653
compro_en_ZONA_7     346
dtype: object

## CONCLUSION

Después de analizar los datos de los clientes contactables que tuvieron un aumento en las compras en 2021 en comparación con 2020, se identificó que la zona con más clientes que cumplen con estas características fue la zona 3. Además, se observó que a estos clientes en particular les gusta comprar productos como camisetas, sandalias, gafas, calcetines y gorras.

En general, el top 3 de productos más populares entre estos clientes en todas las zonas son camisetas, sandalias y gafas.

Con base a esta información, se propone implementar campañas publicitarias en las zonas 3 y 4, promocionando específicamente los productos de camisetas, sandalias y gafas. Esto permitirá aprovechar el interés y la preferencia de compra de estos clientes en estas zonas en particular, con el objetivo de aumentar las ventas y la participación del mercado en esas áreas.

Tambien es importante contactar a las 224 personas que tienen al dia el habeas data, que no estan en la blacklist, que tienen el correo y telefono disponibles y  que no estan satisfechas con los productos que en su mayoria tambien pertenecen a esas zonas y logran entender como se puede mejorar la satisfaccion de los clientes.

Es importante llevar a cabo un análisis continuo de los resultados de estas campañas para evaluar su efectividad y realizar ajustes si es necesario.